# 질문 명사구 변경
`nominal`: (문법) 명사의, 명사적 용법의

In [ ]:
import random
import ast
import pandas as pd
from openai import OpenAI
from google.colab import userdata
import os
import re
from tqdm import tqdm

os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')

In [ ]:
# klue_mrc_context_question_negative_samples.csv
!gdown 1Jy5FX41lSF9VH-LUeCPTgDAn9flXQqHc

Downloading...
From (original): https://drive.google.com/uc?id=1Jy5FX41lSF9VH-LUeCPTgDAn9flXQqHc
From (redirected): https://drive.google.com/uc?id=1Jy5FX41lSF9VH-LUeCPTgDAn9flXQqHc&confirm=t&uuid=2d72e746-ebc8-4f9b-8e12-4029dce96aa4
To: /content/klue_mrc_context_question_negative_samples.csv
100% 161M/161M [00:02<00:00, 59.3MB/s]


In [ ]:
# CSV 파일을 DataFrame으로 읽기
df = pd.read_csv('klue_mrc_context_question_negative_samples.csv')

# 2000~2199번째 행 추출
df_200 = df.iloc[2000:2200].reset_index(drop=True)
df_200

,context,question,negative_samples
0,1일 국회에서 확정된 2014년 예산안에 따르면 올해 한국 국가채무는 지난해보다 2...,2012년 국가채무액은?,['‘최경환 경제팀’이 물려받게 될 나라 살림은 무척 빠듯한 편이다. 재정적자가 갈...
1,마리아나는 비너노이슈타트에서 신성 로마 제국의 황제 페르디난트 3세와 스페인의 마리...,마리아나가 발타사르 카를로스와 결혼한 해는?,"['1751년 나폴리에서 스페인의 카를로스 3세 국왕(나폴리의 카를로 7세, 시칠리..."
2,“자산운용 시스템과 운용인력뿐 아니라 성과(수익률)도 좋았다.”심사위원들이 신한생명...,정량 지표 1위 업체의 정성 평가 등수는?,['제1회 한경수입차서비스지수(KICSI) 평가에서 독일과 일본의 대표적 프리미엄 ...
3,아즈치모모야마 시대\n\n원래 히다 지방은 슈고 교고쿠 가문(京極氏)과 고쿠시(国司...,에마 가문의 멸망은 요리쓰나가 누구와 힘을 합쳐 이루어냈는가?,"[""요리요시가 군세 회복을 위해 힘쓰는 동안 아베씨는 고로모가와 남쪽까지 세력을 넓..."
4,롯데멤버스(대표이사 전형식)가 ‘딥애드(DeepAD)’ 플랫폼에 광고주용 매체 구매...,DSP 플랫폼으로 인해 마케팅 효과의 상승이 기대된다고 한 사람의 이름은?,['“디지털 마케팅 분야는 이제 시장이 막 열린 상태입니다. 이 분야에 커다란 기회...
...,...,...,...
195,1790년대에는 에스테르하지후(候)가 사망하여 후작가의 악단은 해산되고 하이든은 다...,하이든의 말년 작품들에서 가장 돋보이는 성향은?,['마지막 창작 시기의 중심 작품은 비극〈탑〉이었다. 시인은 1920년 여름부터 1...
196,‘중국 기업의 혁신에서 배우자.’샤오미 등 중국 기업의 중저가 공세에 밀려 고전하고...,삼성을 앞지른 중국 기업은?,['중국이 정보기술(IT)산업에서 약진하고 있다. 한국이 주도하던 이동통신기술이나 ...
197,“국내 제약시장의 0.1%에도 미치지 못하는 한약제제 시장에 활력을 불어 넣겠습니다...,한국크라시에가 과립형 제제를 생산하고 있는 지역은?,['‘한라→금강→백두→마칼루→칸첸→K2→에베레스트→퀘이사→아폴로.’SK하이닉스 이천...
198,"서울 송파 생활권인 위례신도시와 경기 화성 동탄2신도시, 김포 한강신도시 등에 중산...",경부고속도로와 가까운 신도시 이름은?,['설 연휴를 맞아 고향 부동산시장을 살펴보는 것은 어떨까. 주변 개발 소식도 들어...


In [ ]:
system_prompt = (
    "당신은 한국어 질문을 더 간결하게 명사구로 바꿔주는 전문가입니다. "
    "아래 지침을 반드시 따라주세요:\n"
    "- 질문의 의미가 절대 손실되지 않도록 합니다.\n"
    "- 질문의 의도를 모두 포함하는 명사구로 바꿔 주세요.\n"
    "- 명사구로 바꿀 수 없는 경우에도 최대한 질문의 요점을 명사구로 변환해 주세요.\n"
    "- 불필요한 설명이나 감탄사는 모두 제거합니다.\n"
    "- 변환 결과만 명사구 한 줄로 출력하세요."
)

user_prompts = [
    f"아래 질문을 명사구로 바꿔 주세요.\n\n질문: {q}"
    for q in df_200['question']
]

In [ ]:
client = OpenAI()
nominal_questions = []

for user_prompt in tqdm(user_prompts, desc="명사구 변환 중"):
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        temperature=0
    )
    nominal = response.choices[0].message.content.strip()
    nominal_questions.append(nominal)

df_200['nominal_question'] = nominal_questions

명사구 변환 중: 100%|██████████| 200/200 [01:45<00:00,  1.89it/s]


In [ ]:
df_200[['question', 'nominal_question']]

,question,nominal_question
0,2012년 국가채무액은?,2012년 국가채무액
1,마리아나가 발타사르 카를로스와 결혼한 해는?,마리아나와 발타사르 카를로스의 결혼 연도
2,정량 지표 1위 업체의 정성 평가 등수는?,정량 지표 1위 업체의 정성 평가 등수
3,에마 가문의 멸망은 요리쓰나가 누구와 힘을 합쳐 이루어냈는가?,에마 가문의 멸망과 요리쓰나의 동맹 상대
4,DSP 플랫폼으로 인해 마케팅 효과의 상승이 기대된다고 한 사람의 이름은?,DSP 플랫폼에 따른 마케팅 효과 상승 기대 발언자 이름
...,...,...
195,하이든의 말년 작품들에서 가장 돋보이는 성향은?,하이든 말년 작품의 돋보이는 성향
196,삼성을 앞지른 중국 기업은?,삼성을 앞지른 중국 기업
197,한국크라시에가 과립형 제제를 생산하고 있는 지역은?,한국크라시에 과립형 제제 생산 지역
198,경부고속도로와 가까운 신도시 이름은?,경부고속도로 인근 신도시 이름


In [ ]:
# 각 row에 대해 docs 리스트를 만들고 DataFrame에 저장
def make_docs_variable(row):
    # negative_samples가 str이면 리스트로 변환
    negatives = row['negative_samples']
    if isinstance(negatives, str):
        negatives = ast.literal_eval(negatives)
    # 오답 개수(0~4개) 랜덤 선택
    n_neg = random.randint(0, min(4, len(negatives)))
    sampled_neg = random.sample(negatives, n_neg) if n_neg > 0 else []
    docs = [row['context']] + sampled_neg
    random.shuffle(docs)
    return docs

df_200['docs'] = df_200.apply(make_docs_variable, axis=1)
df_200[['nominal_question', 'docs']].head()

,nominal_question,docs
0,2012년 국가채무액,[1일 국회에서 확정된 2014년 예산안에 따르면 올해 한국 국가채무는 지난해보다 ...
1,마리아나와 발타사르 카를로스의 결혼 연도,"[1751년 나폴리에서 스페인의 카를로스 3세 국왕(나폴리의 카를로 7세, 시칠리아..."
2,정량 지표 1위 업체의 정성 평가 등수,[제1회 한경수입차서비스지수(KICSI) 평가에서 독일과 일본의 대표적 프리미엄 차...
3,에마 가문의 멸망과 요리쓰나의 동맹 상대,[아버지 데라자와 히로마사(寺沢広政)와 함께 도요토미 히데요시를 섬겼다. 분로쿠 원...
4,DSP 플랫폼에 따른 마케팅 효과 상승 기대 발언자 이름,[롯데멤버스(대표이사 전형식)가 ‘딥애드(DeepAD)’ 플랫폼에 광고주용 매체 구...


In [ ]:
df_200['docs'].apply(len)

,docs
0,2
1,2
2,5
3,5
4,1
...,...
195,2
196,3
197,5
198,5


In [ ]:
df_200['docs'].apply(len).value_counts().sort_index()

,count
docs,
1,40
2,36
3,39
4,37
5,48


In [ ]:
system_prompt = """당신은 주어진 여러 문서(docs)를 바탕으로, 사용자의 질문에 최대한 정확하게, 그리고 문서 내에서만 정보를 근거로 하여 답변하는 AI 비서입니다.
아래 지침을 반드시 지켜주세요:

- 답변은 반드시 docs에서 찾은 내용에 한해서만 작성해주세요. docs에 없는 내용은 추론하거나 지어내지 마세요.
- 답변에서 인용하는 부분이 있다면, 반드시 해당 문서의 번호(예: [[doc1]], [[doc2]])로 근거를 표시해 주세요.
- docs의 순서와 번호는 중요합니다. docs에서 인용하지 않은 정보는 답변에 포함하지 마세요.
- 답변의 근거가 되는 문서 번호를 생략하지 말고, 항상 인용 태그([[doc1]], [[doc2]], ...)를 포함해 주세요.
- 모든 답변은 존댓말을 사용하세요."""

user_prompts = []
system_prompts = []

for idx, row in df_200.iterrows():
    docs = row['docs']  # 이미 섞인 리스트
    # 문서에 번호 붙이기
    doc_items = [f"doc{i+1}: {doc}" for i, doc in enumerate(docs)]
    docs_str = '\n'.join(doc_items)
    # user_prompt 생성
    user_prompt = f"""질문: {row['nominal_question']}

docs:
{docs_str}

위의 docs 중에서만 정보를 근거로 하여, 질문에 답변해 주세요.
답변에서 인용한 문서의 내용에는 반드시 [[doc1]], [[doc2]], ... 형식으로 인용 표시를 해주세요.
추론이나 지어내는 답변은 삼가주시고, docs에 명시적으로 나타난 내용만 인용해 주세요."""
    # 리스트에 추가
    user_prompts.append(user_prompt)
    system_prompts.append(system_prompt)

# 예시 출력
print("system_prompt:")
print(system_prompts[100])
print("\nuser_prompt:")
print(user_prompts[100])

system_prompt:
당신은 주어진 여러 문서(docs)를 바탕으로, 사용자의 질문에 최대한 정확하게, 그리고 문서 내에서만 정보를 근거로 하여 답변하는 AI 비서입니다.
아래 지침을 반드시 지켜주세요:

- 답변은 반드시 docs에서 찾은 내용에 한해서만 작성해주세요. docs에 없는 내용은 추론하거나 지어내지 마세요.
- 답변에서 인용하는 부분이 있다면, 반드시 해당 문서의 번호(예: [[doc1]], [[doc2]])로 근거를 표시해 주세요.
- docs의 순서와 번호는 중요합니다. docs에서 인용하지 않은 정보는 답변에 포함하지 마세요.
- 답변의 근거가 되는 문서 번호를 생략하지 말고, 항상 인용 태그([[doc1]], [[doc2]], ...)를 포함해 주세요.
- 모든 답변은 존댓말을 사용하세요.

user_prompt:
질문: 스티브 랜즈버그의 저서

docs:
doc1: “금융의 시대는 끝났다. 런던의 금융 중심지 시티 오브 런던은 다시 변두리가 될 것이다. 부자가 되고 싶으면 농부가 돼야 한다.”조지 소로스와 함께 전설적 헤지펀드로 유명한 ‘퀀텀펀드’를 만든 짐 로저스는 2010년 영국 옥스퍼드대에서 학생들에게 이같이 조언했다. 그는 “1958년에는 매년 미국에서 배출하는 경영학 석사(MBA)가 5000명이었지만 지금은 20만명이 넘는다”며 “MBA보다 농업과 광업 학위를 따는 것이 더 현명한 선택”이라고 말했다. 세계는 식량 부족에 직면하고 있고 가격은 계속 오르고 있다. 식량 가격이 오르지 않는다면 이를 생산하는 사람이 점차 줄어들 것이고 결국에는 어떤 가격에도 식량을 구할 수 없게 된다는 지적이다.《세계경제의 메가트렌드에 주목하라》는 로저스가 자신의 삶에서 얻은 경험을 바탕으로 경제, 정치, 사회를 분석하고 투자를 위해 조언하는 책이다. 그는 미래 경제는 농업이 주도할 것이라며 식품, 에너지, 상품, 소모품 등을 만드는 사람들이 경제를 이끌 것이라고 말한다. “미국은 터무니없는 교육과 낮은 저축률, 세금 정책, 정치인들의 무능력으

In [ ]:
df_200['user_prompt'] = user_prompts
df_200['system_prompt'] = system_prompts
df_200[['nominal_question', 'docs', 'system_prompt', 'user_prompt']].head()

,nominal_question,docs,system_prompt,user_prompt
0,2012년 국가채무액,[1일 국회에서 확정된 2014년 예산안에 따르면 올해 한국 국가채무는 지난해보다 ...,"당신은 주어진 여러 문서(docs)를 바탕으로, 사용자의 질문에 최대한 정확하게, ...",질문: 2012년 국가채무액\n\ndocs:\ndoc1: 1일 국회에서 확정된 20...
1,마리아나와 발타사르 카를로스의 결혼 연도,"[1751년 나폴리에서 스페인의 카를로스 3세 국왕(나폴리의 카를로 7세, 시칠리아...","당신은 주어진 여러 문서(docs)를 바탕으로, 사용자의 질문에 최대한 정확하게, ...",질문: 마리아나와 발타사르 카를로스의 결혼 연도\n\ndocs:\ndoc1: 175...
2,정량 지표 1위 업체의 정성 평가 등수,[제1회 한경수입차서비스지수(KICSI) 평가에서 독일과 일본의 대표적 프리미엄 차...,"당신은 주어진 여러 문서(docs)를 바탕으로, 사용자의 질문에 최대한 정확하게, ...",질문: 정량 지표 1위 업체의 정성 평가 등수\n\ndocs:\ndoc1: 제1회 ...
3,에마 가문의 멸망과 요리쓰나의 동맹 상대,[아버지 데라자와 히로마사(寺沢広政)와 함께 도요토미 히데요시를 섬겼다. 분로쿠 원...,"당신은 주어진 여러 문서(docs)를 바탕으로, 사용자의 질문에 최대한 정확하게, ...",질문: 에마 가문의 멸망과 요리쓰나의 동맹 상대\n\ndocs:\ndoc1: 아버지...
4,DSP 플랫폼에 따른 마케팅 효과 상승 기대 발언자 이름,[롯데멤버스(대표이사 전형식)가 ‘딥애드(DeepAD)’ 플랫폼에 광고주용 매체 구...,"당신은 주어진 여러 문서(docs)를 바탕으로, 사용자의 질문에 최대한 정확하게, ...",질문: DSP 플랫폼에 따른 마케팅 효과 상승 기대 발언자 이름\n\ndocs:\n...


In [ ]:
client = OpenAI()

# 응답 저장 리스트
responses = []

for i, user_prompt in enumerate(tqdm(user_prompts, desc="응답 생성 중", ncols=80)):
    response = client.chat.completions.create(
        model="gpt-4.1-mini-2025-04-14",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        temperature=0
    )
    answer = response.choices[0].message.content
    responses.append(answer)

응답 생성 중: 100%|███████████████████████████| 200/200 [07:43<00:00,  2.32s/it]


In [ ]:
def extract_doc_citations(text):
    """
    [[doc1]], [[doc2]], ... 형식의 인용 문서번호만 리스트로 반환
    """
    # 정규표현식: 대소문자 구분 없이 [[doc숫자]]
    citations = re.findall(r'\[\[(doc\d+)\]\]', text, flags=re.IGNORECASE)
    return list(set(citations))  # 중복 제거, 원하면 정렬도 가능

In [ ]:
# 1. answer 컬럼 추가
df_200['answer'] = responses

# 2. doc_citations 컬럼 추가
df_200['doc_citations'] = df_200['answer'].apply(extract_doc_citations)

# 3. 결과 일부 확인
df_200[['nominal_question', 'answer', 'docs', 'doc_citations']].head()

,nominal_question,answer,docs,doc_citations
0,2012년 국가채무액,2012년 국가채무액은 443조원으로 기록되었습니다[[doc1]].,[1일 국회에서 확정된 2014년 예산안에 따르면 올해 한국 국가채무는 지난해보다 ...,[doc1]
1,마리아나와 발타사르 카를로스의 결혼 연도,마리아나와 발타사르 카를로스의 결혼 연도에 관한 정보는 docs에 명확히 나와 있지...,"[1751년 나폴리에서 스페인의 카를로스 3세 국왕(나폴리의 카를로 7세, 시칠리아...",[doc2]
2,정량 지표 1위 업체의 정성 평가 등수,"정량 지표 1위 업체는 렉서스이며, 이 업체의 정성 평가 등수는 2위입니다. 문서에...",[제1회 한경수입차서비스지수(KICSI) 평가에서 독일과 일본의 대표적 프리미엄 차...,[doc1]
3,에마 가문의 멸망과 요리쓰나의 동맹 상대,"에마 가문의 멸망에 대해서는, 아즈치모모야마 시대에 미쓰기 가문과 아네코지 가문이 ...",[아버지 데라자와 히로마사(寺沢広政)와 함께 도요토미 히데요시를 섬겼다. 분로쿠 원...,[doc2]
4,DSP 플랫폼에 따른 마케팅 효과 상승 기대 발언자 이름,DSP 플랫폼에 따른 마케팅 효과 상승 기대 발언자는 정란숙 롯데멤버스 데이터애널리...,[롯데멤버스(대표이사 전형식)가 ‘딥애드(DeepAD)’ 플랫폼에 광고주용 매체 구...,[doc1]


In [ ]:
# 인용 문서 정보가 없는 행 제거 (즉, doc_citations가 빈 리스트인 행 제거)
df_with_citation = df_200[df_200['doc_citations'].apply(lambda x: len(x) > 0)].reset_index(drop=True)

print(f"인용 정보가 있는 행만 남은 데이터 shape: {df_with_citation.shape}")

인용 정보가 있는 행만 남은 데이터 shape: (199, 9)


In [ ]:
df_with_citation[['nominal_question', 'docs', 'user_prompt', 'system_prompt', 'answer', 'doc_citations']]

,nominal_question,docs,user_prompt,system_prompt,answer,doc_citations
0,2012년 국가채무액,[1일 국회에서 확정된 2014년 예산안에 따르면 올해 한국 국가채무는 지난해보다 ...,질문: 2012년 국가채무액\n\ndocs:\ndoc1: 1일 국회에서 확정된 20...,"당신은 주어진 여러 문서(docs)를 바탕으로, 사용자의 질문에 최대한 정확하게, ...",2012년 국가채무액은 443조원으로 기록되었습니다[[doc1]].,[doc1]
1,마리아나와 발타사르 카를로스의 결혼 연도,"[1751년 나폴리에서 스페인의 카를로스 3세 국왕(나폴리의 카를로 7세, 시칠리아...",질문: 마리아나와 발타사르 카를로스의 결혼 연도\n\ndocs:\ndoc1: 175...,"당신은 주어진 여러 문서(docs)를 바탕으로, 사용자의 질문에 최대한 정확하게, ...",마리아나와 발타사르 카를로스의 결혼 연도에 관한 정보는 docs에 명확히 나와 있지...,[doc2]
2,정량 지표 1위 업체의 정성 평가 등수,[제1회 한경수입차서비스지수(KICSI) 평가에서 독일과 일본의 대표적 프리미엄 차...,질문: 정량 지표 1위 업체의 정성 평가 등수\n\ndocs:\ndoc1: 제1회 ...,"당신은 주어진 여러 문서(docs)를 바탕으로, 사용자의 질문에 최대한 정확하게, ...","정량 지표 1위 업체는 렉서스이며, 이 업체의 정성 평가 등수는 2위입니다. 문서에...",[doc1]
3,에마 가문의 멸망과 요리쓰나의 동맹 상대,[아버지 데라자와 히로마사(寺沢広政)와 함께 도요토미 히데요시를 섬겼다. 분로쿠 원...,질문: 에마 가문의 멸망과 요리쓰나의 동맹 상대\n\ndocs:\ndoc1: 아버지...,"당신은 주어진 여러 문서(docs)를 바탕으로, 사용자의 질문에 최대한 정확하게, ...","에마 가문의 멸망에 대해서는, 아즈치모모야마 시대에 미쓰기 가문과 아네코지 가문이 ...",[doc2]
4,DSP 플랫폼에 따른 마케팅 효과 상승 기대 발언자 이름,[롯데멤버스(대표이사 전형식)가 ‘딥애드(DeepAD)’ 플랫폼에 광고주용 매체 구...,질문: DSP 플랫폼에 따른 마케팅 효과 상승 기대 발언자 이름\n\ndocs:\n...,"당신은 주어진 여러 문서(docs)를 바탕으로, 사용자의 질문에 최대한 정확하게, ...",DSP 플랫폼에 따른 마케팅 효과 상승 기대 발언자는 정란숙 롯데멤버스 데이터애널리...,[doc1]
...,...,...,...,...,...,...
194,하이든 말년 작품의 돋보이는 성향,[1790년대에는 에스테르하지후(候)가 사망하여 후작가의 악단은 해산되고 하이든은 ...,질문: 하이든 말년 작품의 돋보이는 성향\n\ndocs:\ndoc1: 1790년대에...,"당신은 주어진 여러 문서(docs)를 바탕으로, 사용자의 질문에 최대한 정확하게, ...",하이든 말년 작품의 돋보이는 성향은 다음과 같습니다. 1790년대 후작가 악단 해산...,[doc1]
195,삼성을 앞지른 중국 기업,[중국이 정보기술(IT)산업에서 약진하고 있다. 한국이 주도하던 이동통신기술이나 시...,질문: 삼성을 앞지른 중국 기업\n\ndocs:\ndoc1: 중국이 정보기술(IT)...,"당신은 주어진 여러 문서(docs)를 바탕으로, 사용자의 질문에 최대한 정확하게, ...",삼성을 앞지른 중국 기업으로는 화웨이가 있습니다. 화웨이가 공개한 모바일 애플리케이...,"[doc2, doc1]"
196,한국크라시에 과립형 제제 생산 지역,[‘한라→금강→백두→마칼루→칸첸→K2→에베레스트→퀘이사→아폴로.’SK하이닉스 이천공...,질문: 한국크라시에 과립형 제제 생산 지역\n\ndocs:\ndoc1: ‘한라→금강...,"당신은 주어진 여러 문서(docs)를 바탕으로, 사용자의 질문에 최대한 정확하게, ...",한국크라시에는 현재 중국에서 생약을 끓여 성분을 추출한 뒤 일본 도야마에 있는 크라...,[doc5]
197,경부고속도로 인근 신도시 이름,"[서울 송파 생활권인 위례신도시와 경기 화성 동탄2신도시, 김포 한강신도시 등에 중...",질문: 경부고속도로 인근 신도시 이름\n\ndocs:\ndoc1: 서울 송파 생활권...,"당신은 주어진 여러 문서(docs)를 바탕으로, 사용자의 질문에 최대한 정확하게, ...",경부고속도로 인근 신도시로는 경기 화성 동탄2신도시와 김포 한강신도시가 있습니다. ...,[doc1]


In [ ]:
# 저장할 컬럼 리스트
save_cols = ['nominal_question', 'docs', 'user_prompt', 'system_prompt', 'answer', 'doc_citations']

# 파일로 저장
df_with_citation.to_csv(
    'klue_mrc_nominal_question_docs_1_5_answer_citations.csv',
    columns=save_cols,
    index=False,
    encoding='utf-8'
)

In [ ]:
from google.colab import files
files.download('klue_mrc_nominal_question_docs_1_5_answer_citations.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>